# 2. 파이썬으로 데이터 주무르기, pandas
**pandas를 활용해서 데이터프레임을 다뤄봅시다.**

1. Pandas 시작하기
    - prerequisite : Table
    - pandas import하기
   
2. Pandas로 1차원 데이터 다루기 - Series 
    - Series 선언하기
    - Series vs ndarray
    - Series vs dict
    - Series에 이름 붙이기
3. Pandas로 2차원 데이터 다루기 - dataframe
    - dataframe 선언하기
    - from csv to dataframe
    - dataframe 자료 접근하기

[수업에 사용된 covid 데이터](https://www.kaggle.com/imdevskp/corona-virus-report)

## I. pandas 시작하기

## II. pandas로 1차원 데이터 다루기 - Series

## III. Pandas로 2차원 데이터 다루기 - dataframe

## Mission:
### 1. covid 데이터에서 100 case 대비 사망률(`Deaths / 100 Cases`)이 가장 높은 국가는?

In [1]:
import pandas as pd

# covid = pd.read_csv("./country_wise_latest.csv")
covid = pd.read_csv("/content/drive/MyDrive/programmers_kdt_II/country_wise_latest.csv")

deaths = sorted(covid['Deaths / 100 Cases'])
print(covid[covid['Deaths / 100 Cases'] == deaths[-1]])

    Country/Region  Confirmed  ...  1 week % increase             WHO Region
184          Yemen       1691  ...               4.45  Eastern Mediterranean

[1 rows x 15 columns]


### 2. covid 데이터에서 신규 확진자가 없는 나라 중 WHO Region이 'Europe'를 모두 출력하면?  
Hint : 한 줄에 동시에 두가지 조건을 Apply하는 경우 Warning이 발생할 수 있습니다.

In [2]:
import pandas as pd

# covid = pd.read_csv("./country_wise_latest.csv")
covid = pd.read_csv("/content/drive/MyDrive/programmers_kdt_II/country_wise_latest.csv")

europe = covid[covid['WHO Region'] == 'Europe']
print(europe[europe['New cases'] == 0])

    Country/Region  Confirmed  ...  1 week % increase  WHO Region
56         Estonia       2034  ...               0.64      Europe
75        Holy See         12  ...               0.00      Europe
95          Latvia       1219  ...               2.27      Europe
100  Liechtenstein         86  ...               0.00      Europe
113         Monaco        116  ...               6.42      Europe
143     San Marino        699  ...               0.00      Europe
157          Spain     272421  ...               2.86      Europe

[7 rows x 15 columns]


### 3. 다음 [데이터](https://www.kaggle.com/neuromusic/avocado-prices)를 이용해 각 Region별로 아보카도가 가장 비싼 평균가격(AveragePrice)을 출력하면?

In [3]:
import pandas as pd

# avocado = pd.read_csv("./avocado.csv")
avocado = pd.read_csv("/content/drive/MyDrive/programmers_kdt_II/avocado.csv")

print(avocado['AveragePrice'].groupby(by=avocado['region']).max())

region
Albany                 2.13
Atlanta                2.75
BaltimoreWashington    2.28
Boise                  2.79
Boston                 2.19
BuffaloRochester       2.57
California             2.58
Charlotte              2.83
Chicago                2.30
CincinnatiDayton       2.20
Columbus               2.22
DallasFtWorth          1.90
Denver                 2.16
Detroit                2.08
GrandRapids            2.73
GreatLakes             1.98
HarrisburgScranton     2.27
HartfordSpringfield    2.68
Houston                1.92
Indianapolis           2.10
Jacksonville           2.99
LasVegas               3.03
LosAngeles             2.44
Louisville             2.29
MiamiFtLauderdale      3.05
Midsouth               2.17
Nashville              2.24
NewOrleansMobile       2.32
NewYork                2.65
Northeast              2.31
NorthernNewEngland     1.96
Orlando                2.87
Philadelphia           2.45
PhoenixTucson          2.62
Pittsburgh             1.83
Plains       